<a href="https://colab.research.google.com/github/zhe0/aia_nlp/blob/main/Bertopic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### task: 主題分析、脈絡/框架/議題
##### issue: 或許可找雜訊，主要是外訓有教過就放上來
##### method: BERTopic、distiluse-base-multilingual-cased-v1、sbert/sentence_transformers
##### note: 
* 主題分析加上sentence transformers
* 因為使用中文語料，要斷詞，必須額外確定有中文的sbert以及停用字
* 看是否需要pyldavis做display
* [BERTopic](https://maartengr.github.io/BERTopic/getting_started/quickstart/quickstart.html#keybert-bertopic)
* [sbert](https://www.sbert.net/index.html)

#### 重點前提
* 可以不用指定主題數!!!
* bertopic混合分群、降維、主題分析、tdidf
* 後續圖表有主題機率二維分布圖、降維二維的文章群聚圖(hdbscan and umap)、主題機率時間分析、分類型/屬性的主題機率分布
    * 文章群聚圖:群中心的文章有哪些?離群文章有哪些?
    * 主題在時間比例:觀察主題在不同時間上的比重
    * 主題在分類比例:觀察主題在不同類別上的比重
* 可強化措施
    * 精耕stop word、同義字、剃除雜訊
    * 試試其他embedding
* 加速用，一般colab使用指定device為cuda，請記得開啟colab的ＧＰＵ
    * 執行階段 -> 變更執行階段類型
* 可參考的pretrained model of [sentenceBert](https://www.sbert.net/docs/pretrained_models.html#multi-lingual-models)。找到適合pretrained model請修改sent變數

#### 文本來源
* tvbs, ettoday, ebc, 自由, 聯合 關於維吾爾與新疆新聞文章

In [1]:
# 安裝這些套件
!pip install torch umap-learn scikit-learn bertopic --upgrade
# 下載資料檔(已pickled pandas file)
!gdown 1aDDGuW5IfHbXZ5_vjz2daGotnaFDuBBS
!gdown 1Vitqfhq5gHW9d_cRtEvkXXKh7Ay4ddUM

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 KB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.9/103.9 KB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 52.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.4/662.4 KB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 37.8 MB/s eta 0:00:00
    

In [10]:
# 建議登入google，存檔你的模組檔
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [3]:
import re, pickle, os, torch
import numpy as np
import pandas as pd
import multiprocessing as mp
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
# 偵測一下有無GPU
device='cpu'
if torch.cuda.is_available():
    device='cuda'
elif torch.backends.mps.is_available():
    device='mps'
torch.set_num_threads(mp.cpu_count())
print('your device:',device ,f' and cpu {mp.cpu_count()} cores')

your device: cuda  and cpu 2 cores


In [5]:
# 一些functions
def pkl(path,wbORrb,datas=None):
    '''
    datas:tuples of your data
    wbORr:string only wb or rb
    '''
    if not (wbORrb == 'wb' or wbORrb == 'rb'):
        return None
    if wbORrb == 'rb' and not os.path.exists(path) and datas is not None:
        return None
    try:
        with open(path,wbORrb) as f:
            if wbORrb == 'wb': 
                pickle.dump(datas,f)
                return True
            else:
                return pickle.load(f)
    except Exception as e: 
        print('pickle error:', e)
        return False

# 重要變數
sent='distiluse-base-multilingual-cased-v1' 
# path='/content/drive/MyDrive/school/after/維吾爾/combine_shrink.pkl'
path='/content/combine_shrink.pkl'
d,p=pkl(path,'rb')
d=pd.DataFrame(d)
print('文本總量',len(docs))
d 

文本總量 14637


,media,title,content,time,tok_stopped_str,time_over
0,TVBS,迪麗熱巴、肖戰甜蜜現身機場？狗仔爆料遭揭P圖 粉絲怒了,"30歲大陸維吾爾族女星迪麗熱巴,以甜美不做作個性深得粉絲喜愛,不僅戲劇以及綜藝節目表現亮眼,...",2023/01/04 08:21,30 歲 大陸 維吾爾族 女星 迪麗熱巴 甜美 做作 個性 深得 粉絲 喜愛 戲劇 綜藝 節...,1
1,TVBS,中共為何會殘暴監控新疆維吾爾族人？ 竟與「藏族」有關,"達賴喇嘛的紀錄電影《夢迴西藏:軍官與達賴喇嘛》,除了揭穿他當年率領8萬藏民流亡印度的驚滔駭浪...",2022/12/09 21:17,達賴 喇嘛 紀錄 電影 夢 迴 西藏 軍官 達賴 喇嘛 揭穿 當年 率領 8萬 藏民 流亡 ...,1
2,TVBS,英調查點名多家國際車廠 供應鏈恐涉新疆強迫勞動,"英國雪菲爾哈倫大學團隊耗時6個月調查發現,西方汽車大廠與中國新疆地區的維吾爾少數民族侵犯人權...",2022/12/08 13:43,英國 雪菲爾 哈倫 大學 團隊 耗時 月 調查 發現 西方 汽車 大廠 中國 新疆 地區 維...,1
3,TVBS,「白紙革命」續燒海外！ 土耳其維族人燒習近平像,"「白紙革命」在大陸風波暫歇,然而海外的聲援抗議,正接力登場。包括日本、南韓、美國到歐洲,都有...",2022/12/01 22:25,「 白紙 革命 大陸 風波 暫歇 海外 聲援 抗議 正 接力 登場 包括 日本 南韓 美國 ...,1
4,TVBS,跨族裔支持白紙運動 華府中國使館外嗆中共下台,"反封控「白紙運動」蔓延海外,旅美中國人、維吾爾人與台灣人團結不分你我,今天分頭在華府中國駐美...",2022/12/01 14:45,反 封控 「 白紙 運動 蔓延 海外 旅美 中國人 維吾爾人 台灣人 團結 分 你我 華府 ...,1
...,...,...,...,...,...,...
18042,udn,烏魯木齊（新疆） 棉花去年產量265萬噸 連22年稱冠大陸,"新疆維吾爾自治區農業工作會議指出,2016年新疆棉花播種面積達2,348萬畝,總產量265....",2017-03-02,"新疆 維吾爾 自治區 農業 工作 會議 指出 2016年 新疆 棉花 播種 面積 達 2,3...",1
18043,udn,搶搭一帶一路順風車 華陽集團前進新疆,"搶搭中國一帶一路順風車,華陽集團2015年10月前進新疆自治區,隔年(2016)成立西盟綠能...",2017-02-23,搶搭 中國 一帶 路 順風車 華陽 集團 2015年 10月 新疆 自治區 隔年 (2016...,1
18044,udn,華陽集團簡介,"華陽集團旗下,在台灣有華陽國際企業及凱輝生活實業;在加拿大溫哥華有曄禾公司及日安市集;在中國...",2017-02-23,華陽 集團 旗下 台灣 華陽 國際 企業 凱輝 生活 實業 加拿大 溫哥華 曄禾 公司 日安...,1
18045,udn,熱比婭3月訪台 陸委會將審慎評估,"維吾爾人權領袖熱比婭可望訪台?台灣團結聯盟昨開記者會表示,熱比婭已答應邀請,預計三月底訪台;...",2017-01-25,維吾爾 人權 領袖 熱比婭 可望 訪 台 台灣 團結 聯盟 昨 開 記者會 熱比婭 答應 邀...,1


In [ ]:
# 「

In [ ]:
timestamps=d['time'].tolist()
docs=d['tok_stopped_str'].tolist() 
sentences=d['content'].tolist()
classes=d['media'].tolist()

# 2017後各家報紙數
print(d['media'].value_counts())
# 聲量分析

ltn        7361
ETtoday    3530
udn        2096
TVBS        957
EBC         693
Name: media, dtype: int64

### 建立兩個model，以下p參數可以試試
* normalize_embeddings=True 若樣本量有接近母數可用
* batch_size 若有獨顯可以自由加大，colab就差不多
* low_memory 記憶體不夠請True
* top_n_words 建議放大，上限30
* n_gram_range 若trigram，在後面定義主題意義時發現沒三字，建議回bi

In [6]:
sentence_model = SentenceTransformer(sent,device=device)
p={'sentences':sentences, 'show_progress_bar':False,'device':device,'normalize_embeddings':True,'batch_size':2048}
embeddings = sentence_model.encode(**p)

In [ ]:
# paragraph embedding
sentence_model = SentenceTransformer(sent,device=device)
p={'sentences':sentences, 'show_progress_bar':False,'device':device,'normalize_embeddings':True,'batch_size':4096}
embeddings = sentence_model.encode(**p)
# diversity有助於拉開主題間距哩，但不建議使用，0-1，1拉最開
# low_memory:True有效降低mem使用，但速度會慢
# n_gram_range:基本使用是(1,1)，越多參考越多連續字詞(我使用bi-gram)、trigram用過後沒發現好處
p={'low_memory':False, 'top_n_words': 30,'calculate_probabilities':True, 'verbose':True,
   'n_gram_range':(1,2),'language':'multilingual'}#'embedding_model':sent
topic_model = BERTopic(**p)

In [ ]:
# fit_transform dataset
topics, probs = topic_model.fit_transform(docs,embeddings)

2023-02-01 10:03:09,582 - BERTopic - Reduced dimensionality
2023-02-01 10:03:33,337 - BERTopic - Clustered reduced embeddings


In [9]:
# 儲存與讀取model
# pre=r'/content/drive/MyDrive/school/after/維吾爾/' #請自改
pre=r'/content/' #請自改
mdl_path=pre+'uyghur_topic_mdl'
# topic_model.save(mdl_path)
topic_model = BERTopic.load(mdl_path)

In [ ]:
len(topic_model.topic_sizes_) #主題數

209

In [ ]:
# build時間分析用的model
topics_over_time = topic_model.topics_over_time(docs, timestamps=timestamps)

3730it [2:33:06,  2.46s/it]


In [ ]:
# 儲存與讀取time model
mdl_time_path=pre+'uyghur_topic_time.pd'
pkl(mdl_time_path,'wb',datas=topics_over_time)

#### 右側主題可以互動，點擊後可以選擇或不選擇，觀察主題的時間變化，也可觀察主題內的用詞
* topics 以list:int為element，指定你要show的主題。topic_model.topic_sizes_就全部都show
* normalize_frequency 好像沒效...? 建議先給False

In [ ]:
p={'topics_over_time':topics_over_time,'normalize_frequency':False,
   'topics':[i for i in range(7+1)]+[-1],}
topic_model.visualize_topics_over_time(**p)

### 主觀賦予主題意義
* 針對某主題的topN關鍵字， topN數量取決於top_n_words，無法在多

In [ ]:
topic_model.get_topic(7)

[('教育營', 0.015083381245002546),
 ('教育', 0.014896341729242651),
 ('新疆', 0.011410570337120828),
 ('培訓', 0.010566034027172636),
 ('關押', 0.009463173885556857),
 ('學員', 0.007945696340321261),
 ('職業', 0.007719603692871741),
 ('技能', 0.007399206273986536),
 ('中心', 0.007086473869089562),
 ('培訓 中心', 0.006989935749387867),
 ('職業 技能', 0.006985621167994994),
 ('穆斯林', 0.0067733336256696535),
 ('教育 培訓', 0.00665729295766382),
 ('當局', 0.00647986494712798),
 ('文件', 0.005894689975429584),
 ('民族', 0.005821464022126134),
 ('技能 教育', 0.005664649852311873),
 ('中國 政府', 0.005550946492307634),
 ('報導', 0.005535191507847865),
 ('恐怖', 0.00548149212515199),
 ('新疆 教育營', 0.005295453562914128),
 ('維吾爾', 0.005260370774042834),
 ('維吾爾人', 0.0050984491664858925),
 ('政府', 0.005079337839195398),
 ('思想', 0.004871614839283369),
 ('主義', 0.004629033734989432),
 ('維吾爾族', 0.004549164282529353),
 ('學校', 0.004494835324010649),
 ('中國', 0.004416589452426547),
 ('報導 中國', 0.004394209167560295)]

In [ ]:
# 僅是用圖例展示斷詞站主題的機率，幫助主觀賦以主題意義。前5主題的斷詞分佈
topic_model.visualize_barchart([i for i in range(5)])

In [ ]:
# 若不好猜回主題，可看主題在模組建立時，參考哪三篇文章最貼近
# 假設我看不懂第一個主題
topic_idx=1
[i for i in topic_model.get_representative_docs(topic_idx)[:2]] #只取第一二篇

['記者 潘泓鈺 澳門 報導   寶島 夢想家 57 94 不敵 新疆 飛虎 李學林 陳世念 狀況 輸球 年輕 球隊 成長 必 路 期盼 未來 越 越   寶島 夢想家 出征 澳門 超級8 夏季 聯賽 18日 李學林 碰上 老東家 CBA 新疆 飛虎 可惜 拿下 勝利 最終 57 94 吞下 二 連敗   李學林 內容 理想 輸球 昨天 狀況 第一 場 面對 菲律賓 PBA 聯盟 公路 勇士 太 精神 陳世念 認為 感覺 放開 空間 希望 未來 場 場   「 夢想家 隻 年輕 球隊 經驗 輸球 成長 必 路 李學林 說道 「 趁著 吸取 經驗 教訓   新疆 飛虎 次 帶 青年隊 出征 包含 台灣 義守 大學 白曜誠 透過 超級 偶像 李學林 陳世念 「 打球 真的 對到 真的 緊張 超級 機會 抱持 著 挑戰 心態 好好 享受',
 '記者 潘泓鈺 綜合 報導   季 中國 CBA 一共 名 台將 創下 歷史 新高 表現 亮眼 33 歲 年紀 打出 生涯 年 楊敬敏 漸入佳境 趨勢 稍嫌 可惜 展現 狀元 身手 陳盈駿 受傷 報銷 榜眼 周儀翔 沒 節奏 僅 場 得分 上雙   林志傑 2009年 加盟 浙江 廣廈 為 台灣 球員 開啟 中國 CBA 道 大門 去年 胡瓏貿 成為 第一 位 選秀會 挑中 台灣 球員 季 更 陳盈駿 周儀翔 包辦 首 輪 前 兩 順位 年紀 增長 像是 曾文鼎 李學林 歷經 角色 轉換 楊敬敏 身 出 歲月 侵蝕 23 場 比賽 10 場 得分 超過 16 分 更 2017年 「 封關戰 豪取 生涯 新高 33 分 場 均 14.3 分 籃板 3.1 助攻 打出 西進 賽季 最佳 成績   「 阿美 戰士 楊敬敏 2012年 加盟 山西 汾酒 上海 大鯊魚 曾文鼎 攜手 去年 轉戰 北控 翱龍 位 台將 中 數 耀眼 位 CBA 得分 排行榜 前 20 名 場 均 26.5 分 排名 第20 名 丁彥宇航 外援 楊敬敏 14.3 分 排 非 外援 第11 位 場均 籃板 3.1 助攻 1.4 抄截 生涯 新高 得分 生涯 次 佳   中國 球迷 封為 「 廣廈隊 魂 林志傑 球隊 公認 老大哥 開季 單 場 11 分 籃板 助攻 全能 成績單 可惜 傷 缺席 場 比賽 生涯 首度 場 均 得分 未 達 雙位數 林志傑 復出 場 比

In [ ]:
# qry給關鍵字，若關鍵字全中的文章秀出10篇。幫忙一下回找重點字詞的文章
qry=['國務院','眾議院','法案']
idx=[i for i in range(len(docs)) if all([True if j in docs[i] else False for j in qry])][:10]
d.iloc[idx][['content']]

,content
58,"聯邦眾議院今天針對中國打壓維吾爾穆斯林少數民族,以壓倒性票數通過法案,限制自新疆進口商品。議..."
101,"美國眾議院去年以壓倒性票數,通過《防止強迫維吾爾勞動法》後,該法案卻遲遲沒有在參議院過關,美..."
216,"在美國國務卿龐佩奧與中共中央政治局委員楊潔篪會談之際,美國總統川普今天正式簽署維吾爾人權法案..."
235,"「國際調查記者聯盟」上個月取得大陸官方文件,踢爆北京在新疆再教育營中,關押跟懲罰囚犯的細節,..."
510,"陸媒觀察者網報導,英特爾(Intel)近日寫信給供應商表示,將確保英特爾供應鏈不使用任何來自..."
837,"共機今天(9/24)再度入侵台灣西南跟北部空域,八天內第七度擾台。大陸解放軍,也從今天起到九..."
1509,眾議院的院會於2月4日表決通過美國競爭法案(America COMPETES Act of ...
1694,"文/中央社記者徐薇婷華盛頓25日專電 國際抵制北京2022冬奧聲浪劇增,美國眾院外委會今天通..."
1818,"美國國務卿蓬佩奧(Michael Pompeo)6月26日宣布,將對破壞香港高度自治和人權的..."
1831,"文/中央社記者江今葉華盛頓17日專電 在美國國務卿蓬佩奧與中共中央政治局委員楊潔篪會談之際,..."


### 用途:你觀察到有興趣的文章，可能是群中心、可能是離群文章、可能是你注重的長報導、可能是注重的高轉發高討論高按讚帖子
* approximate_distribution、approximate_distribution執行一次就好
* 文章順序，改idx當參數

In [ ]:
topic_distr, _ = topic_model.approximate_distribution(docs) #角度是documental embedding
topic_token_distr, token_distr = topic_model.approximate_distribution(docs, calculate_tokens=True)
topic_token_distr.shape,topic_distr.shape #這matrix代表說，(文章或斷詞)對應主題機率組合，單篇文章(內的文章或斷詞)加總必為百。

100%|██████████| 15/15 [01:22<00:00,  5.51s/it]


((14637, 208), (14637, 208))

In [ ]:
doc_idx=192
topic_model.visualize_distribution(topic_distr[doc_idx]) #該篇文章的主題機率組合

In [ ]:
# 看看第十篇文章是因為那些字詞被分到這12個主題
doc_idx=192
df = topic_model.visualize_approximate_distribution(docs[doc_idx], token_distr[doc_idx])
df

In [ ]:
# 用途1.看相對位置與圓大小，確認training效果好不好。
# 用途2.建議主題數要少才會有對比效果，或直接指定某幾個主題看看距離，但只是看個感覺...
topic_model.visualize_topics()

### 資料內有分類屬性的話，可試試。用途於不同分類的主題量(累積主題機率)差別
* topics 選擇你要比較的主題

In [ ]:
topics_per_class = topic_model.topics_per_class(docs, classes=classes)

5it [00:23,  4.62s/it]


In [ ]:
# 儲存與讀取cls model
mdl_cls_path=pre+'uyghur_topic_cls.pd'
pkl(mdl_cls_path,'wb',datas=topics_per_class)

True

In [ ]:
topic_model.visualize_topics_per_class(topics_per_class,topics=[i for i in range(5)])

### 將斷詞慢慢分階層，觀察字詞如何細分到各個主題，有幫助於建立抽象的大主題

In [ ]:
hierarchical_topics = topic_model.hierarchical_topics(docs)
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

100%|██████████| 218/218 [00:06<00:00, 33.41it/s]


In [20]:
# 是以doc當作一個點，並畫出圖，用途是找群中心的文章(找標準)，以及遠離群的文章(找異常)，以及人工確認群間的分界(soft clustering)
# 限制:因為降成兩維，很多資訊會被壓縮，可能在三維或多維空間才是正常主題距離，當個參考就好

# Run the visualization with the original embeddings
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine',low_memory=False).fit_transform(embeddings)

In [38]:
# colab 免費版記憶體不足，畫不出來，必須要local電腦才能繪製...
topic_model.visualize_documents(docs,reduced_embeddings=reduced_embeddings)

In [41]:
pkl('tmp.pkl','wb',datas=(docs,reduced_embeddings))

True

#### 以下幾個ｃｅｌｌ為備選步驟，可以嘗試看看

In [ ]:
# 三個進階用法
# 使用時機：當主題數太多時候使用，避免過多分析

# 法一：reduce topics，好處是保留原始由演算法自行決定的結果
# topic_model.reduce_topics(c, nr_topics=10) #保留10主題 (或說壓縮到10主題)
# len(topic_model.topics_),topic_model.get_topic(0)

# 法二：研究者自己決定合併主題，下面例子是一二合併、三四合併
# topics_to_merge = [[1, 2]
#                     [3, 4]]
# topic_model.merge_topics(c, topics_to_merge)

# 法三: Fine-tune topic representations after training BERTopic
# 備選步驟可不做，嘗試的話，可用於2次精煉結果
# vectorizer_model = CountVectorizer(stop_words="english", ngram_range=(1, 3), min_df=1)
# topic_model.update_topics(c, vectorizer_model=vectorizer_model)

In [ ]:
# # 接續上面，可對umap加速，但是卻少更多特徵
# from sklearn.decomposition import PCA
# pca_embeddings = PCA(n_components=2).fit_transform(embeddings) 
# reduced_embeddings = UMAP(n_neighbors=10, n_components=2, metric='cosine',low_memory=False,init=pca_embeddings).fit_transform(embeddings)
# topic_model.visualize_documents(doc, reduced_embeddings=reduced_embeddings)

In [ ]:
# 雖然叫做reduce_outliers，經嘗試後，文章只留下一個主題
new_topics = topic_model.reduce_outliers(docs, topics, strategy="probabilities",probabilities=probs)#probabilities=probs if probabilities is used
new_topics[10] #第10篇文章的主題

NameError: ignored

In [ ]:
# 使用reduce_outliers別種參數，文章依舊只留下一個主題
new_topics = topic_model.reduce_outliers(c, topics, strategy="distributions")
new_topics[10] #第10篇文章的主題

In [ ]:
# 太多embedding可以選擇... 自行嘗試

# SentenceTransformer
# from sentence_transformers import SentenceTransformer
# sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
# topic_model = BERTopic(embedding_model=sentence_model)

# spaCy
# import spacy
# # from thinc.api import set_gpu_allocator, require_gpu  #gpu version
# # require_gpu(0) #gpu version
# nlp = spacy.load("en_core_web_md", exclude=['tagger', 'parser', 'ner', 'attribute_ruler', 'lemmatizer'])
# topic_model = BERTopic(embedding_model=nlp)

# Hugging Face Transformers
# from transformers.pipelines import pipeline
# embedding_model = pipeline("feature-extraction", model="distilbert-base-cased")
# topic_model = BERTopic(embedding_model=embedding_model)

# Universal Sentence Encoder (USE)
# import tensorflow_hub
# embedding_model = tensorflow_hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
# topic_model = BERTopic(embedding_model=embedding_model)

# gensim
# import gensim.downloader as api
# ft = api.load('fasttext-wiki-news-subwords-300')
# topic_model = BERTopic(embedding_model=ft)

# sklearn TruncatedSVD
# from sklearn.pipeline import make_pipeline
# from sklearn.decomposition import TruncatedSVD
# from sklearn.feature_extraction.text import TfidfVectorizer
# pipe = make_pipeline(TfidfVectorizer(),TruncatedSVD(100))
# topic_model = BERTopic(embedding_model=pipe)

# Word + Document Embeddings
from bertopic.backend import WordDocEmbedder
import gensim.downloader as api
from sentence_transformers import SentenceTransformer
ft = api.load('fasttext-wiki-news-subwords-300') # Word embedding model
embedding_model = SentenceTransformer(sent) # Document embedding model。中文paraphrase-multilingual-mpnet-base-v2
# all-MiniLM-L6-v2備註: Intended uses Our model is intented to be used as a sentence and short paragraph encoder.
# Create a model that uses both language models and pass it through BERTopic
word_doc_embedder = WordDocEmbedder(embedding_model=embedding_model, word_embedding_model=ft)
topic_model = BERTopic(embedding_model=word_doc_embedder)

[==================================================] 100.0% 958.5/958.4MB downloaded


#### memo

In [ ]:
# 可存成html
# h=topic_model.visualize_documents(doc, reduced_embeddings=reduced_embeddings).to_html()
# h=topic_model.visualize_topics().to_html()
with open('dummy.html','w') as p:
    p.write(h)

In [ ]:
# stop words
# removing stop words as a preprocessing step is not advised
# As the transformer-based embedding models that we use need the full context to create accurate embeddings.
# 建議
# After having generated embeddings and clustered our documents. 
# I have found almost no disadvantages to using the CountVectorizer to remove stop words
from sklearn.feature_extraction.text import CountVectorizer
stopwords=['the','is','you']
vectorizer_model = CountVectorizer(stop_words=stopwords)
topic_model = BERTopic(vectorizer_model=vectorizer_model)

# 抑制高頻詞頻
# 不能與上面stop_words一起通用
from bertopic.vectorizers import ClassTfidfTransformer
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)
topic_model = BERTopic(ctfidf_model=ctfidf_model)

In [ ]:
# easy start
# embedding_model當語言不同務必修改支援語系
# topic_model = BERTopic(low_memory=True,embedding_model=sent)
# topics, probs = topic_model.fit_transform(docs)
topic_model.get_topic_info() #綜觀summary結果

,Topic,Count,Name
0,-1,4561,-1_中國_美國_台灣_新疆
1,0,638,0_cba_籃板_助攻_球隊
2,1,433,1_迪麗熱巴_女星_網友_粉絲
3,2,336,2_冬奧_北京 冬奧_奧運_抵制
4,3,302,3_香港_國安法_港人_民主
...,...,...,...
215,214,11,214_棉花_播種_棉花 播種_優質 商品
216,215,11,215_沃爾瑪_山姆_會員店_山姆 會員店
217,216,10,216_外國人_華春瑩_棉花_鍵盤
218,217,10,217_張菲_辛曉琪_張韶涵_faye


In [ ]:
# words = topic_model.vectorizer_model.get_feature_names()
# tfidf=topic_model.c_tf_idf_
# print('topic model 利用字頻於文章的 tfidf分數，總計有',len(words),' 這些詞彙貢獻在主題的分數',tfidf.shape)
# print('假設棉花在各個主題的貢獻度是',tfidf[words.index('棉花')])
# 
a=topic_model.c_tf_idf_.toarray()
a[:,words.index('棉花')] #順序不知對不對，因為機率很小